# PyCity Schools Analysis : Farah Clerveau
### Import, Merge & Explore Data

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv("Resources/schools_complete.csv")
student_data = pd.read_csv("Resources/students_complete.csv")

In [4]:
#Explore School Data to see what the fields are and their data types
school_data.info()
school_data.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
School ID      15 non-null int64
school_name    15 non-null object
type           15 non-null object
size           15 non-null int64
budget         15 non-null int64
dtypes: int64(3), object(2)
memory usage: 680.0+ bytes


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411


In [5]:
#Explore Student Data to see what the fields are 
student_data.info()
student_data.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 7 columns):
Student ID       39170 non-null int64
student_name     39170 non-null object
gender           39170 non-null object
grade            39170 non-null object
school_name      39170 non-null object
reading_score    39170 non-null int64
math_score       39170 non-null int64
dtypes: int64(3), object(4)
memory usage: 2.1+ MB


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61


In [6]:
# We see that school name is the common factor so we will merge on that column

combo_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
combo_data.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


### Task 1: District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
#Use school_data to gather information about total budget and total schools 

total_budget = school_data["budget"].sum()
total_schools = school_data["school_name"].count()
total_students = school_data["size"].sum()

#Use student_data gather information about student count, average math score, average reading score
avgread = student_data["reading_score"].mean()
avgmath = student_data["math_score"].mean()

# Passing grades are considered to be 70 and above 
passing_read = student_data.loc[student_data["reading_score"] >= 70]
passing_read.sort_values("reading_score") # checking to make sure the lowest number in reading score is 70
pct_read = round((passing_read["reading_score"].count() / total_students)*100, 6)

passing_math = student_data.loc[student_data["math_score"] >= 70]
pct_math = round((passing_math["math_score"].count() / total_students)*100, 6)

#overall passing rate is avg of average math and average read vs. avg of the passing scores 
pct_ov = ((avgmath + avgread)/2) 


# We'll create a new Pandas DataFrame using dictionaries 

d = {"Total Schools": [total_schools], "Total Students": "{:,}".format(total_students)
, "Total Budget": "${:,.2f}".format(total_budget), "Average Math Score" : [avgmath],
                                "Average Reading Score" : [avgread], "% Passing Math" :[pct_math],
                                "% Passing Reading": [pct_read],"Overall Passing Rate": [pct_ov]}

district_summary =pd.DataFrame(data =d)

district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


### Task 2: School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [38]:
combo_ss = combo_data
combo_ss = combo_ss.loc[:, ["school_name", "type", "size", "budget", 
                              "reading_score", "math_score"]]

combo_ss['Passed Math'] = [ 1 if x >= 70 else 0 for x in combo_ss['math_score']]
combo_ss['Passed Read'] = [ 1 if x >= 70 else 0 for x in combo_ss['reading_score']]

combo_ss = combo_ss.rename(columns={"type":"School Type", "size": "Total Students",
                                   "budget": "Total School Budget", 
                                    "reading_score":"Average Reading Score",
                                   "math_score":"Average Math Score"})
ss_groupby = combo_ss.groupby("school_name").agg({"Total Students":"mean", "Total School Budget":"mean",
                                                    "Average Reading Score" : "mean",
                                                   "Average Math Score": "mean", 
                                                    "Passed Math" : "sum",
                                                   "Passed Read" : "sum"})

ss_groupby['% Passing Math'] = (ss_groupby['Passed Math'] / ss_groupby['Total Students'])*100
ss_groupby['% Passing Reading'] = (ss_groupby['Passed Read'] / ss_groupby['Total Students'])*100
ss_groupby['% Overall Passing Rate'] = (ss_groupby['% Passing Math'] + ss_groupby['% Passing Reading'])/ 2
ss_groupby["Per Student Budget"] = ss_groupby['Total School Budget']  / ss_groupby['Total Students']

add_schooltype = pd.merge(ss_groupby, school_data, how="left", on=["school_name", "school_name"])
ss_groupby2 = add_schooltype[["school_name", "type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score",
                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
ss_groupby2 = ss_groupby2.rename(columns={"type": "School Type", "school_name":"School Name"})

ss_groupby2["Total Students"] = ss_groupby2["Total Students"].astype(int).map("{:,}".format)
ss_groupby2["Total School Budget"] = ss_groupby2["Total School Budget"].astype(int).map("${:,.2f}".format)
ss_groupby2["Per Student Budget"] = ss_groupby2["Per Student Budget"].astype(int).map("${:,.2f}".format)

ss_groupby2

ss_groupby2.set_index('School Name', inplace=True)
ss_groupby2.index.name = None

ss_groupby2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


### Task 3: Top Performing Schools (By Passing Rate)

* Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [40]:
top_school = ss_groupby2.sort_values(["% Overall Passing Rate"], ascending = False).head(5)

top_school.index.name = None 
top_school

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


### Task 4: Bottom Performing Schools (By Passing Rate)

* Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [41]:
bottom_school = ss_groupby2.sort_values(["% Overall Passing Rate"], ascending = True).head(5)
bottom_school.index.name = None 
bottom_school

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


### Task 5: Math Scores by Grade\*\*

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [33]:
math_grades = combo_data.loc[:, ["school_name","grade", "math_score"]]


math_grades_nine = math_grades.loc[math_grades["grade"] == "9th"]
math_grades_nine["9th"] = math_grades_nine["math_score"]
math_grades_nine= math_grades_nine.groupby("school_name").mean()
math_grades_nine2 = math_grades_nine["9th"]


math_grades_ten = math_grades.loc[math_grades["grade"] == "10th"]
math_grades_ten["10th"] = math_grades_ten["math_score"]
math_grades_ten= math_grades_ten.groupby("school_name").mean()
math_grades_ten2 = math_grades_ten["10th"]


math_grades_eleven = math_grades.loc[math_grades["grade"] == "11th"]
math_grades_eleven["11th"] = math_grades_eleven["math_score"]
math_grades_eleven= math_grades_eleven.groupby("school_name").mean()
math_grades_eleven2 = math_grades_eleven["11th"]


math_grades_tweleve = math_grades.loc[math_grades["grade"] == "12th"]
math_grades_tweleve["12th"] = math_grades_tweleve["math_score"]
math_grades_tweleve= math_grades_tweleve.groupby("school_name").mean()
math_grades_tweleve2 = math_grades_tweleve["12th"]


math_subset=pd.DataFrame(columns = ["9th", "10th", "11th", "12th"])
math_subset["9th"] = math_grades_nine2
math_subset["10th"] = math_grades_ten2
math_subset["11th"] = math_grades_eleven2
math_subset["12th"] = math_grades_tweleve2

math_subset.index.name = None

math_subset

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


### Task 6: Reading Scores by Grade\*\*

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [35]:
read_grades = combo_data.loc[:, ["school_name","grade", "reading_score"]]


read_grades_nine = read_grades.loc[read_grades["grade"] == "9th"]
read_grades_nine["9th"] = read_grades_nine["reading_score"]
read_grades_nine= read_grades_nine.groupby("school_name").mean()
read_grades_nine2 = read_grades_nine["9th"]


read_grades_ten = read_grades.loc[read_grades["grade"] == "10th"]
read_grades_ten["10th"] = read_grades_ten["reading_score"]
read_grades_ten= read_grades_ten.groupby("school_name").mean()
read_grades_ten2 = read_grades_ten["10th"]


read_grades_eleven = read_grades.loc[read_grades["grade"] == "11th"]
read_grades_eleven["11th"] = read_grades_eleven["reading_score"]
read_grades_eleven= read_grades_eleven.groupby("school_name").mean()
read_grades_eleven2 = read_grades_eleven["11th"]


read_grades_tweleve = read_grades.loc[read_grades["grade"] == "12th"]
read_grades_tweleve["12th"] = read_grades_tweleve["reading_score"]
read_grades_tweleve= read_grades_tweleve.groupby("school_name").mean()
read_grades_tweleve2 = read_grades_tweleve["12th"]


reading_subset=pd.DataFrame(columns = ["9th", "10th", "11th", "12th"])
reading_subset["9th"] = read_grades_nine2
reading_subset["10th"] = read_grades_ten2
reading_subset["11th"] = read_grades_eleven2
reading_subset["12th"] = read_grades_tweleve2

reading_subset.index.name = None

reading_subset

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


### Task 7: Scores by School Spending
* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. 

In [29]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

ss_groupby_b = add_schooltype[["school_name", "type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score",
                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
ss_groupby_b = ss_groupby_b.rename(columns={"type": "School Type", "school_name":"School Name"})


ss_groupby_b["Spending Ranges (Per Student)"] = pd.cut(ss_groupby_b["Per Student Budget"], spending_bins, labels=group_names, right=True)

budget_bin = ss_groupby_b[["Spending Ranges (Per Student)", "Average Math Score",
                          "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
budget_bin_gb= budget_bin.groupby("Spending Ranges (Per Student)").mean()
budget_bin_gb

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


### Task 8: Scores by School Size

In [31]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

schoolsize_b = add_schooltype[["school_name", "type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score",
                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
schoolsize_b = schoolsize_b.rename(columns={"type": "School Type", "school_name":"School Name"})

schoolsize_b["School Size"] = pd.cut(schoolsize_b["Total Students"], size_bins, labels=group_names, right=True)
schoolsize_bin = schoolsize_b[["School Size", "Average Math Score",
                          "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
schoolsize_bin_gb= schoolsize_bin.groupby("School Size").mean()
schoolsize_bin_gb

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


### Task 9: Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [32]:
ss_groupby_sch = ss_groupby2[["School Type","Average Math Score",
                                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
type_groupby = ss_groupby_sch.groupby("School Type")


type_groupby.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Observable Trends
* Schools that had more Per Student Budget did not necessarily do better. 
* Mid-Sized Schools (those with between 1000-2000 students) had the highest % Passing Rates in Math and Reading. The majority of Charter schools fall within this range except for Wilson High School with 2,283 students, Pena High School with 962 students, and Holden High School with 427 students. All of the district schools have over 2,000 students. 